In [51]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report
import random

Использую готовый датасет с [kaggle](https://www.kaggle.com/datasets/neisse/scrapped-lyrics-from-6-genres?select=lyrics-data.csv) , удаляю пустые строки

In [127]:
songs = pd.read_csv("lyrics-data.csv")
songs = songs.dropna()
songs

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt
...,...,...,...,...,...
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en


Очищаю имя автора

In [128]:
def repl(x):
    args = x.split('/')
    if len(args) == 1:
        return args[0]
    return args[1]

songs['ALink'] = songs['ALink'].apply(repl)
songs = songs.drop('language', axis=1)
songs = songs.drop('SLink', axis=1)
songs = songs.drop('SName', axis=1)
songs

,ALink,Lyric
0,ivete-sangalo,"Tudo o que eu quero nessa vida,\nToda vida, é\..."
1,ivete-sangalo,Meu coração\nSem direção\nVoando só por voar\n...
2,ivete-sangalo,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...
3,ivete-sangalo,Quando a chuva passar\n\nPra quê falar\nSe voc...
4,ivete-sangalo,A minha sorte grande foi você cair do céu\nMin...
...,...,...
379926,clegg-johnny,Chorus\nHere we stand waiting on the plain\nDa...
379927,clegg-johnny,I nearly disappeared into the mouth of a croco...
379928,clegg-johnny,"Amambuka, amambuka azothengisa izwe lakithi, i..."
379929,clegg-johnny,Sweat in the heat for days on end\nwaiting for...


In [129]:
print(f"Количество артистов: {songs['ALink'].nunique()}")

Количество артистов: 4225


Присваиваю каждому автору уникальный номер

In [130]:
names = songs['ALink'].unique()
ids = {}
for i in range(len(names)):
    ids[names[i]] = i + 1

ids

{'ivete-sangalo': 1,
 'chiclete-com-banana': 2,
 'banda-eva': 3,
 'e-o-tchan': 4,
 'claudia-leitte': 5,
 'harmonia-do-samba': 6,
 'ara-ketu': 7,
 'daniela-mercury': 8,
 'olodum': 9,
 'netinho': 10,
 'asa-de-aguia': 11,
 'cheiro-de-amor': 12,
 'timbalada': 13,
 'carlinhos-brown': 14,
 'tomate': 15,
 'jammil-e-uma-noites': 16,
 'luiz-caldas': 17,
 'margareth-menezes': 18,
 'jauperi': 19,
 'babado-novo': 20,
 'parangole': 21,
 'psirico': 22,
 'banda-grafith': 23,
 'tchakabum': 24,
 'banda-mel': 25,
 'black-style': 26,
 'chicabana': 27,
 'banda-beijo': 28,
 'axe-bahia': 29,
 'oz-bambaz': 30,
 'rapazolla': 31,
 'tonho-materia': 32,
 'alexandre-peixe': 33,
 'leva-noiz': 34,
 'lucas-fellix': 35,
 'saiddy-bamba': 36,
 'sharon-axe-moi': 37,
 'talokudo': 38,
 'ta-demais': 39,
 'samir-matos': 40,
 'ouro-fino': 41,
 'gab-ferruz': 42,
 'cannibal': 43,
 'uata-anderson': 44,
 'beyonce': 45,
 'bruno-mars': 46,
 'rihanna': 47,
 '50-cent': 48,
 'john-legend': 49,
 'racionais-mcs': 50,
 'michael-jackson'

Заменяю в датасете имена авторов на их номера

In [131]:
def replids(x):
    return ids[x]

songs['ALink'] = songs['ALink'].apply(replids)
songs

,ALink,Lyric
0,1,"Tudo o que eu quero nessa vida,\nToda vida, é\..."
1,1,Meu coração\nSem direção\nVoando só por voar\n...
2,1,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...
3,1,Quando a chuva passar\n\nPra quê falar\nSe voc...
4,1,A minha sorte grande foi você cair do céu\nMin...
...,...,...
379926,4225,Chorus\nHere we stand waiting on the plain\nDa...
379927,4225,I nearly disappeared into the mouth of a croco...
379928,4225,"Amambuka, amambuka azothengisa izwe lakithi, i..."
379929,4225,Sweat in the heat for days on end\nwaiting for...


Очищаю текст песен от всех символов кроме букв, цифр, пробелов

In [132]:
def repltext(x):
    regex = re.compile('[\s]')
    x = str(regex.sub(' ', x))
    regex = re.compile('[^a-zA-Z0-9 ]')
    x = str(regex.sub('', x))
    # regex = re.compile('[^]')
    return x

songs['Lyric'] = songs['Lyric'].apply(repltext)
songs

,ALink,Lyric
0,1,Tudo o que eu quero nessa vida Toda vida ama...
1,1,Meu corao Sem direo Voando s por voar Sem sabe...
2,1,de babaix de balacubaca de babaix de balac...
3,1,Quando a chuva passar Pra qu falar Se voc no ...
4,1,A minha sorte grande foi voc cair do cu Minha ...
...,...,...
379926,4225,Chorus Here we stand waiting on the plain Dark...
379927,4225,I nearly disappeared into the mouth of a croco...
379928,4225,Amambuka amambuka azothengisa izwe lakithi izw...
379929,4225,Sweat in the heat for days on end waiting for ...


Произвожу кластеризацию

In [146]:
df = songs.copy()
df = df.loc[:1000]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Lyric'])

num_clusters = df['ALink'].nunique()
kmeans = KMeans(n_clusters=num_clusters, n_init=10)
kmeans.fit(tfidf_matrix)

df['cluster'] = kmeans.labels_
print(accuracy_score(df['ALink'], df['cluster']))
print(classification_report(df['ALink'], df['cluster'], zero_division=0))

0.305668016194332
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.29      0.37      0.32       312
           2       0.27      0.42      0.33       260
           3       0.20      0.04      0.06       211
           4       0.71      0.55      0.62       129
           5       0.00      0.00      0.00        76

    accuracy                           0.31       988
   macro avg       0.25      0.23      0.22       988
weighted avg       0.30      0.31      0.28       988



Пытаюсь в цикле перебрать некоторое множество возможных значений для n_init, запускаю кластеризацию на большем объёме данных, смотрю, при каких значениях точность выше

In [152]:
df = songs.copy()
df = df.loc[:3000]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Lyric'])

acc = []
for i in range(1, 20):
    num_clusters = df['ALink'].nunique()
    kmeans = KMeans(n_clusters=num_clusters, n_init=i)
    kmeans.fit(tfidf_matrix)

    df['cluster'] = kmeans.labels_
    acc.append(accuracy_score(df['ALink'], df['cluster']))

for i in range(len(acc)):
    print("init({:<2}) -> accuracy({})".format(i, acc[i]))

init(0 ) -> accuracy(0.059399257509281135)
init(1 ) -> accuracy(0.07897401282483969)
init(2 ) -> accuracy(0.056699291258859266)
init(3 ) -> accuracy(0.05231184610192373)
init(4 ) -> accuracy(0.05028687141410732)
init(5 ) -> accuracy(0.03914951063111711)
init(6 ) -> accuracy(0.0931488356395545)
init(7 ) -> accuracy(0.06986162672966588)
init(8 ) -> accuracy(0.062099223759703004)
init(9 ) -> accuracy(0.045899426257171784)
init(10) -> accuracy(0.08842389470131623)
init(11) -> accuracy(0.067161660479244)
init(12) -> accuracy(0.06749915626054674)
init(13) -> accuracy(0.09044886938913263)
init(14) -> accuracy(0.060411744853189335)
init(15) -> accuracy(0.05433682078974013)
init(16) -> accuracy(0.06344920688491394)
init(17) -> accuracy(0.06277421532230847)
init(18) -> accuracy(0.045224434694566315)


Не трудно заметить в большинстве случаев модель плохо сопоставляет текст песни с исполнителем, вполне логично, что стиль написания у многих исполнителей трудно будет отличить